# Sectional lung analysis

Aim:
- Divide lungs into equally spaced sections
- Analyze distribution in each section
    - Histograms and pdfs
    - Test of distribution
- Compare sections in each patient - does any section stand out?

In [1]:
import os
import numpy as np
import re

from preprocess_tools import *

/Users/bruger/anaconda3/envs/bachelor/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Read segmentations

In [106]:
def make_output_dirs(output_folder_dir, patient_ids):
    output_dirs = []
    for patient_id in patient_ids:
        output_dirs.append(output_folder_dir + f"/_{patient_id}_" + '.nii.gz')
    return output_dirs

def get_sorted_dirs(dirs, patient_ids):
    patient_file_map = {pid: [] for pid in patient_ids}

    for dir in dirs:
        # Extract patient ID from the filename
        patient_id = re.search(r'(_\d{4}_)', os.path.basename(dir)).group().split('_')[1]
        if patient_id in patient_file_map:
            patient_file_map[patient_id].append(dir)

    # Sort the patient IDs
    sorted_patient_ids = sorted(patient_ids, key=int)

    # Create a sorted list of file paths based on sorted patient IDs
    sorted_ct_dirs = []
    for pid in sorted_patient_ids:
        sorted_ct_dirs.extend(patient_file_map[pid])
    
    return sorted_ct_dirs

In [94]:
def get_patient_ids(input_dir, file_name_startswith):
    patient_ids = []
    for file_name in os.listdir(input_dir):
        if file_name.startswith(file_name_startswith) and file_name.endswith('.nii.gz'):
            patient_id = re.search(r'(_\d{4}_)', file_name).group().split('_')[1]
            patient_ids.append(patient_id)
    return patient_ids

def get_file_dirs(folder_dir, file_name_startswith):
    file_paths = []
    for file_name in os.listdir(folder_dir):
        if file_name.startswith(file_name_startswith) and file_name.endswith('.nii.gz'):
            file_paths.append(os.path.join(folder_dir, file_name))
    return file_paths

def get_files(file_paths):
    files = []
    for file_path in file_paths:
        # file_path = os.path(file_path)
        file_as_np = load_nifti_convert_to_numpy(input_path=file_path)
        files.append(file_as_np)
    return files

def get_lung_seg(total_segs):
    lung_segs = []
    for total_seg in total_segs:
        lung_segs.append(np.isin(total_seg,np.array([10,11,12,13,14])).astype(int))
    return lung_segs

In [67]:
ct_folder_dir = '/Users/bruger/Desktop/Bachelor/data/ct_images_segmented'
seg_folder_dir = '/Users/bruger/Desktop/Bachelor/data/segmentations'

In [68]:
patient_ids = get_patient_ids(input_dir=seg_folder_dir, file_name_startswith='total_seg_Lung-Pilot-1')

In [69]:
ct_dirs = get_file_dirs(folder_dir=ct_folder_dir, file_name_startswith='Lung-Pilot-1')
ct_dirs = get_sorted_dirs(ct_dirs, patient_ids)

In [95]:
total_seg_dirs = get_file_dirs(folder_dir=seg_folder_dir, file_name_startswith='total_seg_Lung-Pilot-1')
total_seg_dirs = get_sorted_dirs(total_seg_dirs, patient_ids)

In [96]:
total_segs = get_files(file_paths=total_seg_dirs)

In [98]:
lung_segs = get_lung_seg(total_segs)

In [107]:
output_dirs = make_output_dirs('/Users/bruger/Desktop/Bachelor/data/lung_segmentations', patient_ids)
output_dirs = get_sorted_dirs(output_dirs, patient_ids)

In [108]:
# Save lung segmentations for inspection
for lung_seg, output_dir, ct_dir in zip(lung_segs, output_dirs, ct_dirs):
    convert_numpy_to_nifti_and_save(np_file=lung_seg, output_path=output_dir, original_nifti_path=ct_dir)

saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving


# Divide segmentations into more sections
- Find min and max position of segmentation and divide into 10(?)
- Maybe divide in right and left lung - 20 segments

In [114]:
def get_segmentation_layer_range(segmentations):
    seg_ranges = []

    for segmentation in segmentations:
        seg_bottom = None
        seg_top = None
        seg_size = None

        for i in range(segmentation.shape[2]):
            layer = segmentation[:, :, i]
            if seg_bottom is None and np.count_nonzero(layer) > 0:
                seg_bottom = i

            reversed_i = segmentation.shape[2] - 1 - i
            rev_layer = segmentation[:, :, reversed_i]
            if seg_top is None and np.count_nonzero(rev_layer) > 0:
                seg_top = reversed_i

            if seg_bottom is not None and seg_top is not None:
                seg_size = seg_top - seg_bottom
                break

        seg_ranges.append([seg_bottom, seg_top, seg_size])

    return seg_ranges

def get_sections(lung_seg_ranges, n_sections=10):
    """
    Returns which layers (z-axis) in which the lungs should be divided into, to be equally divided into n_sections
    """
    sections = []
    for lung_seg_range in lung_seg_ranges:
        seg_bottom, _, seg_size = lung_seg_range
        sec_size = int(seg_size / n_sections)

        section_start = []
        for i in range(n_sections):
            section_start.append(seg_bottom + i*sec_size)
        sections.append(section_start)
    return sections

In [112]:
lung_seg_ranges = get_segmentation_layer_range(lung_segs)

In [124]:
sections = get_sections(lung_seg_ranges)

Del segmentation op efter sections

# Apply segments to CT images

# Plots and statistical analysis